In [3]:
!pip install tensorflow

In [4]:
import tensorflow as tf

In [5]:
!pip install transformers

In [6]:
import transformers

In [7]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

In [8]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv("./spam.csv", encoding = "ISO-8859-1")

In [11]:
df.head(5)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [12]:
df.shape

(5572, 5)

In [13]:
df.rename(columns = {"v1":"label", "v2":"input"}, inplace=True)

In [15]:
df = df[["label", "input"]]

In [16]:
df.head(2)

,label,input
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...


In [17]:
X = list(df["input"])
y = list(df["label"])

In [18]:
y = list(pd.get_dummies(y, drop_first=True)["spam"])

In [19]:
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)

C:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [20]:
tokenizer("hello this is demo token")

{'input_ids': [101, 7592, 2023, 2003, 9703, 19204, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [21]:
model = TFDistilBertForSequenceClassification.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [23]:
type(X_train)

list

In [24]:
train_encoding = tokenizer(X_train, truncation=True, padding=True, max_length=128)
test_encoding = tokenizer(X_test, truncation=True, padding=True, max_length=128)

In [25]:
type(train_encoding)

transformers.tokenization_utils_base.BatchEncoding

In [26]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encoding),
    y_train,
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encoding),
    y_test,
))

In [27]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(128,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(128,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.bool, name=None))>

In [28]:
type(train_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(8), validation_data=test_dataset.batch(8), epochs=3)